In [1]:
import os
import pickle
import numpy as np
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from autogen_core.models import UserMessage
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core.model_context import BufferedChatCompletionContext

/home/deep/Desktop/Abdullah/autogen_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import faiss


# ✅ Use the smaller BGE model for faster encoding
embedding_model = SentenceTransformer('BAAI/bge-small-en-v1.5')

PDF_DIR = "/home/deep/Desktop/Abdullah/autogen_env/pdfs"
EMBEDDINGS_FILE = "/home/deep/Desktop/Abdullah/autogen_env/data/pdf_embeddings.pkl"

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

def chunk_text(text, max_length=500):
    words = text.split()
    return [" ".join(words[i:i + max_length]) for i in range(0, len(words), max_length)]

def embed_and_store():
    documents = []
    embeddings = []

    for filename in os.listdir(PDF_DIR):
        if filename.endswith(".pdf"):
            print(f"📄 Processing: {filename}")
            text = extract_text_from_pdf(os.path.join(PDF_DIR, filename))
            for chunk in chunk_text(text):
                chunk = chunk.strip()
                if not chunk:
                    continue
                try:
                    vector = embedding_model.encode(chunk, normalize_embeddings=True)
                    vector = np.array(vector, dtype='float32')

                    if vector.ndim == 1 and np.isfinite(vector).all():
                        documents.append((filename, chunk))
                        embeddings.append(vector)
                except Exception as e:
                    print(f"⚠️ Skipping problematic chunk in {filename}: {e}")

    if not embeddings:
        print("❌ No valid embeddings generated.")
        return

    embeddings_np = np.vstack(embeddings).astype('float32')
    index = faiss.IndexFlatIP(embeddings_np.shape[1])
    index.add(embeddings_np)

    with open(EMBEDDINGS_FILE, 'wb') as f:
        pickle.dump({"index": index, "documents": documents}, f)

    print(f"✅ Saved {len(documents)} chunks and their embeddings to {EMBEDDINGS_FILE}")

embed_and_store()

📄 Processing: Standards of Care in Diabetes—2023.pdf
📄 Processing: Diabetes-Medicines.pdf
✅ Saved 13 chunks and their embeddings to /home/deep/Desktop/Abdullah/autogen_env/data/pdf_embeddings.pkl


In [8]:
with open(EMBEDDINGS_FILE, 'rb') as f:
    data = pickle.load(f)

index = data["index"]        # 🧠 Smart index (FAISS)
documents = data["documents"]  # 📄 [(filename, chunk), ...]


def search_similar_chunks(query: str, top_k: int = 30) -> dict[str, list[str]]:
    # Convert question to vector
    query_vec = embedding_model.encode(query, normalize_embeddings=True).astype('float32')
    
    # Search the most similar chunks
    D, I = index.search(np.array([query_vec]), top_k)

    # Collect the matching chunks from their files
    retrieved_chunks = {}
    for i in I[0]:
        filename, chunk = documents[i]
        retrieved_chunks.setdefault(filename, []).append(chunk)
    
    return retrieved_chunks

In [12]:
ollama_client = OllamaChatCompletionClient(model="deepseek-r1:70b")

import asyncio
from autogen_core.models import UserMessage

async def test_ollama():
    result = await ollama_client.create([UserMessage(content="What is the capital of Saudi Arabia?", source="user")])
    print(result)

await test_ollama()

finish_reason='stop' content="<think>\nOkay, so I need to find out what the capital of Saudi Arabia is. Hmm, I'm not entirely sure, but I think it's a city that starts with an 'R'. Maybe Riyadh? I've heard that name before in the context of Saudi Arabia. But wait, could it be somewhere else?\n\nLet me think about other cities in Saudi Arabia. There's Jeddah, which is a major port city, and Mecca, which is really important for religious reasons, but I don't think either of those are capitals. Capitals are usually the administrative centers, so Riyadh sounds more like it because I've heard it mentioned in news related to the government or the royal family.\n\nWait, does Saudi Arabia have any other cities that could be capitals? There's also Medina and Dammam, but those don't ring a bell as capitals. No, I think Riyadh is the right answer. It's where the king lives, right? And most government buildings are there.\n\nI'm pretty confident now that Riyadh is the capital of Saudi Arabia.\n</t

In [20]:
re_ranker_agent = AssistantAgent(
    name="re_ranker",
    model_client=ollama_client,
    system_message=(
        "You are a medical document re-ranking expert.\n"
        "Given a user question and a list of document chunks, return the most relevant 3–5 chunks "
        "that directly answer the question. Remove irrelevant or general content."
    ),
    model_context=BufferedChatCompletionContext(buffer_size=5),
)

validator_agent = AssistantAgent(
    name="validator",
    model_client=ollama_client,
    system_message=(
        "You are a medical question-answer validator.\n"
        "You will receive a question and a chunk of text.\n"
        "Answer ONLY: 'YES - [short reason]' or 'NO - [short reason]'."
    ),
    model_context=BufferedChatCompletionContext(buffer_size=3),
)


In [21]:
query = "What Meglitinides medicines and how they work?" #CHANGE QUESTION HERE

retrieved_chunks_by_file = search_similar_chunks(query)
flat_chunks = [f"From {f}:\n{chunk}" for f, chunks in retrieved_chunks_by_file.items() for chunk in chunks]

In [22]:
re_rank_prompt = f"Question: {query}\n\nChunks:\n\n" + "\n\n".join(flat_chunks)
re_ranked = await re_ranker_agent.run(task=re_rank_prompt)
top_chunks = re_ranked.messages[-1].content.strip()

In [30]:
for i, chunk in enumerate(top_chunks.split("\n\n")):
    check_prompt = f"Question: {query}\n\nChunk:\n{chunk.strip()}"
    verdict = await validator_agent.run(task=check_prompt)
    response = verdict.messages[-1].content.strip()

    print(f"\n🔎 Chunk #{i+1}")
    print("🧪 Validator:", response)
    print("📄 Preview:", chunk[:200], "...")

    if "YES" in response.upper():
        validated_chunks.append(chunk)


🔎 Chunk #1
🧪 Validator: <think>
Alright, let's break this down. The user is asking about Meglitinides again, but the chunk they've provided is empty this time. From previous interactions, I know that when there was a relevant chunk, it mentioned PrandiMet containing repaglinide, which is a Meglitinide.

But now, with no text in the chunk, there's nothing to extract any information from. So, the answer has to reflect that the necessary details aren't present because there's no content provided.
</think>

Answer: NO - The chunk does not provide information about Meglitinides medicines or their mechanism of action.
📄 Preview: <think>
Okay, so I need to figure out the answer to the question "What are the different types of diabetes medications?" based on the provided PDF snippets. Let me go through each part step by step. ...

🔎 Chunk #2
🧪 Validator: <think>
Alright, let me try to figure this out. The user is asking about Meglitinides again, specifically what they are and how they work. L

In [31]:
print(f"Number of validated chunks: {len(validated_chunks)}")

Number of validated chunks: 5


In [32]:
final_answer = " ".join([chunk.strip() for chunk in validated_chunks])
print("🩺 Final Combined Answer:\n")
print(final_answer)

🩺 Final Combined Answer:

1. SGLT2 Inhibitors
2. DPP-4 Inhibitors
3. GLP-1 Receptor Agonists
4. Biguanides (Metformin)
5. Sulfonylureas
6. Thiazolidinediones
7. Alpha-glucosidase Inhibitors
8. Meglitinides
9. Amylin Analogues
10. Combination Medications 4. **Biguanides**: Metformin is the primary medication in this class. 5. **Sulfonylureas**: Medications like glyburide, glimepiride, and glipizide fall into this category. 8. **Meglitinides**: These are rapid-acting insulin secretagogues, such as repaglinide (Prandin) and nateglinide (Starlix). 10. **Combination Medications**: These combine different classes to provide comprehensive management, such as ActoPlus Met (pioglitazone and metformin) and PrandiMet (repaglinide and metformin).
